In [8]:
import sys
sys.path.append('../../personal_pkgs')

from korean_crawling_bundles import *

from konlpy.tag import *
okt = Okt()
import datetime
from IPython.display import display

import re
import numpy as np
import pandas as pd
from pprint import pprint
from konlpy.tag import Hannanum

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

stopwords는 불필요한 단어, 즉 조사나 관사들을 없애는 툴이다.

In [11]:
stop_words = list(pd.read_csv('../../personal_pkgs/kor_stop_words.csv').T.iloc[:1,:].values[0])
stop_words[:10]

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']

# 전체 데이터프레임 병합하기

In [12]:
naver_df = pd.read_csv('../../folder/'+'{}_with_link_naver.csv'.format(str(datetime.datetime.now().month) + str(datetime.datetime.now().day),index=False))
daum_df = pd.read_csv('../../folder/'+'{}_with_link_daum.csv'.format(str(datetime.datetime.now().month) + str(datetime.datetime.now().day),index=False))
daum_df = daum_df.loc[:,['title','link','content']]

df = pd.concat([naver_df,daum_df],axis=0)
df.reset_index(drop=True,inplace=True)
df.tail()

,title,link,content
772,"'9·13 대책의 위력'..""8·2 뛰어넘고 금융위기 수준 육박""",http://m.finance.daum.net/m/news/news_content....,(서울=뉴스1) 국종환 기자 = 고강도 세금·대출 규제인 9·13 부동산대책이 주택...
773,"'럭키박스'에 빠진 유통가..""잡음 있어도 잘 팔리니 '럭키'""",http://m.finance.daum.net/m/news/news_content....,[데일리안 = 손현진 기자]유통업계가 브랜드나 가격이 제각각인 상품을 무작위로 담아...
774,외국인 소유 땅 여의도 84배.. 공시가 30조2820억원,http://m.finance.daum.net/m/news/news_content....,외국인이 보유한 국내 토지 면적이 여의도 면적의 약 84배 금액은 30조원이 넘는 ...
775,중국인이 사들이는 한국 땅 갈수록 줄어,http://m.finance.daum.net/m/news/news_content....,(세종=연합뉴스) 윤종석 기자 = 중국인의 우리나라 토지 보유 증가율이 해마다 줄어...
776,"5G 첫 전파, 하루 앞으로..통신사 불꽃 경쟁 '예고'",http://m.finance.daum.net/m/news/news_content....,【서울=뉴시스】최선윤 기자 = 5G 상용화가 하루 앞으로 다가온 가운데 통신3사가 ...


# 병합한 데이터 프레임에서 중복되는 기사 내용 없애주기

In [13]:
append_ls,append_idx = [],[]
for idx,i in enumerate(df['title']) : 
    if i not in append_ls:
        append_ls.append(i)
        append_idx.append(idx)

In [14]:
df = df.loc[append_idx]
# df.reset_index(drop=True,inplace=True)
df.tail()

,title,link,content
742,반도체 내리막에 스마트폰까지 꺾여..불안한 삼성,http://m.finance.daum.net/m/news/news_content....,◆ 삼성의 위기감 ◆ 1987년 17조4000억원이었던 그룹 매출은 2017년 31...
743,BMW화재 유탄맞은 국내 車산업..5배 징벌적 배상 폭탄,http://m.finance.daum.net/m/news/news_content....,◆ 한국경제 車쇼크 / ⑨ 제조사 옭아매는 '차관법' 개정 ◆하지만 A사는 결함이 ...
744,"""가짜 일자리에 혈세 못 쓴다""..국회 예산심사서 줄줄이 삭감 예고",http://m.finance.daum.net/m/news/news_content....,[ 임도원/박종필 기자 ] 국회 예산심사에서 정부 일자리 창출 사업이 줄줄이 칼질...
745,'대도시 광역교통 콘트롤타워' 광역교통위원회 내년 3월 설립,http://m.finance.daum.net/m/news/news_content....,(세종=연합뉴스) 윤종석 기자 = 국토교통부는 '대도시권광역교통위원회' 설립 근거를...
746,공공건축물 디자인 품격 높인다..'건축서비스산업진흥법' 국회 통과,http://m.finance.daum.net/m/news/news_content....,【서울=뉴시스】최희정 기자 = 공공건축물의 건축기획 개념을 명문화하고 업무절차를 규...


In [15]:
df.fillna('0',inplace=True)
#결측치를 0으로 바꿔준다.

In [16]:
df['content'] = [i.replace('  ','') for i in df['content'].values]

________

# Tuning_process

In [17]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

# 뒤에 부분의 저자에 관한 내용을 가능한 만큼 잘라준다.
data = [val[:re.search('Copyrights|ⓒ|▶',val).start()] if ('Copyrights' in val) or ('ⓒ' in val) or ("▶" in val) else val for val in data]

# 대괄호 안에 저자의 이름이 나와있는 경우를 없애준다.
data = [re.sub('\[.+\]','',val) for val in data]

pprint(data[0])

(' 한국은행이 1년 만에 기준금리를 0.25%포인트 인상하면서 시중은행의 예·적금 금리도 일제히 상승할 것으로 전망된다. 30일 금융권에 '
 '따르면 우리은행은 다음달 3일부터 적금 상품 31개와 정기예금 상품 16개의 금리를 0.1~0.3%포인트 인상한다. 예를 들어 '
 '‘위비Super 주거래 적금Ⅱ’는 최고 연 2.4%에서 최고 연 2.7%로 ‘우리 첫거래 감사적금’은 최고 연 3.0%에서 최고 연 '
 '3.2%로 ‘위비Super 주거래 예금Ⅱ’는 최고 연 2.1%에서 최고 연 2.4%로 각각 오른다.신한은행과 KEB하나은행·NH농협은행도 '
 '다음달부터 예·적금 금리를 0.1∼0.3%포인트 올린다. KB국민은행은 다음달 6일부터 0.25%포인트 수준에서 금리를 인상하고 정기예금 '
 '외에 각종 수신 상품 금리도 순차적으로 높일 계획이다.인터넷전문은행은 시중은행보다 더 큰 폭의 예·적금 금리 인상에 나선다. '
 '한국카카오은행은 1일부터 예·적금 상품의 금리를 최대 0.50%포인트 인상한다. 카카오뱅크 정기예금의 금리는 만기 기간에 관계 없이 '
 '0.30%포인트 오른다. 6개월 만기 정기예금의 금리는 1.80%에서 2.10%로 오르고 1년 만기 정기예금의 금리는 2.20%에서 '
 '2.50%로 인상된다. 1년 만기 적금의 금리는 현재 2.00%에서 2.50%로 2년 만기는 2.10%에서 2.60%로 0.5%포인트씩 '
 '오른다. 은행 관계자는 “기준금리 인상 결정에 따라 시장금리 상승에 신속하게 대응하기 위한 차원”이라고 설명했다. 한편 예·적금 금리가 '
 '오르면 다음달 코픽스(COFIX·자금조달비용지수) 금리도 이를 반영하게 돼 차주의 대출금리 상승 부담은 커질 수 있다. /황정원기자 ')


In [9]:
from pykospacing import spacing

Using TensorFlow backend.


In [10]:
import warnings
warnings.filterwarnings("error")

In [11]:
%%time

spacing_ls = []

for idx,val in enumerate(data) : 
    if idx % 100 ==0 : print('{}th index is done!'.format(idx))
    try : 
         spacing_ls.append(','.join([spacing(val) for val in val.split(".")]).replace(",",''))
    except : spacing_ls.append(val)

0th index is done!
100th index is done!
200th index is done!
300th index is done!
400th index is done!
500th index is done!
600th index is done!
700th index is done!
CPU times: user 8min 56s, sys: 2min 11s, total: 11min 7s
Wall time: 2min 43s


In [12]:
data = spacing_ls.copy()

# tokenization process

- 정규식 표현을 통해서 문장 내에 이메일과 기타 특수 문자들을 없애주었지만, 여전히 난잡해보인다.
- LDA 알고리즘을 사용하기 위해서는, 문장들을 단어들의 묶음으로 변환시켜주는 과정이 필요하다.
- 이러한 과정을 Tokenization 이라고 한다.

In [18]:
%time data_words = [okt.nouns(val) for val in data]
print(data_words[:1])

CPU times: user 26.7 s, sys: 217 ms, total: 26.9 s
Wall time: 17.5 s
[['한국은행', '기준금리', '포인트', '인상', '시중', '은행', '예', '적금', '금리', '일제', '상승', '것', '전망', '금융', '우리은행', '다음', '달', '적금', '상품', '개', '정기예금', '상품', '개', '금리', '포인트', '인상', '예', '위비', '주거', '적금', '최고', '연', '최고', '연', '로', '우리', '거래', '감사', '적금', '은', '최고', '연', '최고', '연', '로', '위비', '주거', '예금', '최고', '연', '최고', '연', '로', '각각', '신한은행', '하나은행', '농협', '은행', '다음', '달', '예', '적금', '금리', '포인트', '올린다', '국민은행', '다음', '달', '포인트', '수준', '금리', '인상', '정기예금', '외', '각종', '수신', '상품', '금리', '순차', '계획', '인터넷', '은행', '시중', '은행', '더', '폭', '예', '적금', '금리', '인상', '한국', '카카오', '은행', '예', '적금', '상품', '금리', '최대', '포인트', '인상', '카카오', '뱅크', '정기예금', '금리', '만기', '기간', '관계', '포인트', '개월', '만기', '정기예금', '금리', '로', '만기', '정기예금', '금리', '로', '인상', '만기', '적금', '금리', '현재', '로', '만기', '로', '포인트', '은행', '관계자', '기준금리', '인상', '결정', '시장', '금리', '상승', '대응', '위', '차원', '설명', '한편', '예', '적금', '금리', '다음', '달', '코픽스', '자금', '조달', '비용', '지수', '금리', '반영', '차주', '대출', '금리',

In [19]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  
        # deacc=True removes punctuations
        # 구두점(말끝에 찍는 쉼표나 점들을 의미) 을 없애주는 것이다.

gensim_data_words = list(sent_to_words(data))

print(gensim_data_words[:1])

[['한국은행이', '만에', '기준금리를', '포인트', '인상하면서', '시중은행의', '적금', '금리도', '일제히', '상승할', '것으로', '전망된다', '금융권에', '따르면', '우리은행은', '다음달', '일부터', '적금', '상품', '개와', '정기예금', '상품', '개의', '금리를', '포인트', '인상한다', '예를', '들어', '위비super', '주거래', '적금ⅱ', '최고', '에서', '최고', '우리', '첫거래', '감사적금', '최고', '에서', '최고', '위비super', '주거래', '예금ⅱ', '최고', '에서', '최고', '각각', '오른다', '신한은행과', 'keb하나은행', 'nh농협은행도', '다음달부터', '적금', '금리를', '포인트', '올린다', 'kb국민은행은', '다음달', '일부터', '포인트', '수준에서', '금리를', '인상하고', '정기예금', '외에', '각종', '수신', '상품', '금리도', '순차적으로', '높일', '계획이다', '인터넷전문은행은', '시중은행보다', '폭의', '적금', '금리', '인상에', '나선다', '한국카카오은행은', '일부터', '적금', '상품의', '금리를', '최대', '포인트', '인상한다', '카카오뱅크', '정기예금의', '금리는', '만기', '기간에', '관계', '없이', '포인트', '오른다', '개월', '만기', '정기예금의', '금리는', '에서', '오르고', '만기', '정기예금의', '금리는', '에서', '인상된다', '만기', '적금의', '금리는', '현재', '에서', '만기는', '에서', '포인트씩', '오른다', '은행', '관계자는', '기준금리', '인상', '결정에', '따라', '시장금리', '상승에', '신속하게', '대응하기', '위한', '차원', '이라고', '설명했다', '한편', '적금', '금리가', '오르면', '다음달', '코픽스', 'cofix', '자금조달비용지수', 

### Bigram , Trigram 모델 만들기
- Bigram : 문서에서 함께 자주 등장하는 2개의 단어
- Trigram : 문서에서 함께 자주 등장하는 3개의 단어
- ‘front_bumper’, ‘oil_leak’, ‘maryland_college_park’ etc.
- Phrases : 모델을 빌드한다.
- min_count , threshold : Pharases 의 중요한 두 개의 파라미터
    - min_count (float, optional) : Ignore all words and bigrams with total collected count lower than this value.
    - threshold (float, optional) : Represent a score threshold for forming the phrases (higher means fewer phrases). A phrase of words a followed by b is accepted if the score of the phrase is greater than threshold. Heavily depends on concrete scoring-function, see the scoring parameter.

In [20]:
warnings.filterwarnings("ignore")

In [21]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['한국은행', '기준금리', '포인트', '인상', '시중_은행', '예_적금', '금리', '일제', '상승', '것', '전망', '금융', '우리은행', '다음_달', '적금', '상품', '개', '정기예금', '상품', '개', '금리', '포인트', '인상', '예', '위비_주거_적금', '최고', '연', '최고', '연', '로', '우리_거래_감사', '적금', '은', '최고', '연', '최고', '연', '로', '위비_주거_예금', '최고', '연', '최고', '연', '로', '각각', '신한은행', '하나은행', '농협', '은행', '다음_달', '예_적금', '금리', '포인트', '올린다', '국민은행', '다음_달', '포인트', '수준', '금리', '인상', '정기예금', '외', '각종', '수신', '상품', '금리', '순차', '계획', '인터넷', '은행', '시중_은행', '더', '폭', '예_적금', '금리', '인상', '한국', '카카오', '은행', '예_적금', '상품', '금리', '최대', '포인트', '인상', '카카오_뱅크', '정기예금', '금리', '만기', '기간', '관계', '포인트', '개월', '만기', '정기예금', '금리', '로', '만기', '정기예금', '금리', '로', '인상', '만기', '적금', '금리', '현재', '로', '만기', '로', '포인트', '은행', '관계자', '기준금리', '인상', '결정', '시장', '금리', '상승', '대응', '위', '차원', '설명', '한편', '예_적금', '금리', '다음_달', '코픽스_자금_조달', '비용', '지수', '금리', '반영', '차주', '대출', '금리', '상승', '부담', '수', '황정', '기자']


In [22]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [23]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

data_lemmatized = data_words_bigrams.copy()

print(data_lemmatized[:1])

[['한국은행', '기준금리', '포인트', '인상', '시중_은행', '적금', '금리', '일제', '상승', '전망', '금융', '우리은행', '적금', '상품', '정기예금', '상품', '금리', '포인트', '인상', '위비_주거', '적금', '최고', '최고', '거래_감사', '적금', '최고', '최고', '위비_주거', '예금', '최고', '최고', '신한은행', '하나은행', '농협', '은행', '적금', '금리', '포인트', '올린다', '국민은행', '포인트', '수준', '금리', '인상', '정기예금', '수신', '상품', '금리', '순차', '계획', '인터넷', '은행', '시중_은행', '적금', '금리', '인상', '한국', '카카오', '은행', '적금', '상품', '금리', '최대', '포인트', '인상', '카카오_뱅크', '정기예금', '금리', '만기', '기간', '관계', '포인트', '개월', '만기', '정기예금', '금리', '만기', '정기예금', '금리', '인상', '만기', '적금', '금리', '현재', '만기', '포인트', '은행', '관계자', '기준금리', '인상', '결정', '시장', '금리', '상승', '대응', '차원', '설명', '한편', '적금', '금리', '코픽스', '자금_조달', '비용', '지수', '금리', '반영', '차주', '대출', '금리', '상승', '부담', '황정', '기자']]


- LDA 모델에 들어가야 하는 두 가지 입력변수는 딕셔너리와(id2word) 코퍼스(corpus)이다.
- gensim 은 문서 내에 있는 단어별로 유니크한 아이디를 할당해준다.
- 아래의 각각의 엘리먼트 튜플당 의미하는 것은 [word_id,word_frequency] 이다.

In [24]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 15), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 5), (16, 1), (17, 1), (18, 1), (19, 3), (20, 4), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 4), (33, 7), (34, 1), (35, 1), (36, 1), (37, 9), (38, 1), (39, 5), (40, 1), (41, 1), (42, 1), (43, 6), (44, 1), (45, 1), (46, 1), (47, 1), (48, 7), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1)]]


In [25]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [26]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.008*"거래" + 0.007*"시장" + 0.006*"기자" + 0.006*"판매" + 0.006*"기준" + 0.005*"보유" '
  '+ 0.005*"업체" + 0.005*"상품" + 0.005*"게임" + 0.005*"제품"'),
 (1,
  '0.027*"그룹" + 0.011*"인사" + 0.010*"승진" + 0.010*"부문" + 0.009*"상무" + 0.009*"은행" '
  '+ 0.008*"임원" + 0.007*"영업" + 0.007*"올해" + 0.006*"보험"'),
 (2,
  '0.047*"금리" + 0.033*"인상" + 0.014*"대출" + 0.014*"기준금리" + 0.011*"한국은행" + '
  '0.011*"증가" + 0.009*"포인트" + 0.009*"경기" + 0.009*"경제" + 0.009*"전망"'),
 (3,
  '0.013*"중국" + 0.010*"사업" + 0.008*"기업" + 0.008*"한국" + 0.007*"산업" + '
  '0.006*"자동차" + 0.006*"조사" + 0.006*"지원" + 0.006*"미국" + 0.005*"안전"'),
 (4,
  '0.022*"카드" + 0.011*"회장" + 0.009*"금융" + 0.007*"업계" + 0.007*"정부" + '
  '0.007*"수수료" + 0.007*"결제" + 0.006*"가맹" + 0.006*"소비자" + 0.005*"혜택"')]


In [27]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.306468758846732

Coherence Score:  0.41291209441735843


In [29]:
mallet_path = '/home/hskimim/Documents/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=5, id2word=id2word)

In [30]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet , corpus=corpus, texts=data)

#ldamallet 파라미터를 optimal_params 로 바꿔주는 시퀀스로 업데이트할 것!

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2.0,0.6190,"금리, 인상, 기준금리, 포인트, 대출, 내년, 금융, 경기, 시장, 한국은행",한국은행이 1년 만에 기준금리를 0.25%포인트 인상하면서 시중은행의 예·적금 금...
1,1,2.0,0.5387,"금리, 인상, 기준금리, 포인트, 대출, 내년, 금융, 경기, 시장, 한국은행",한국은행이 기준금리를 0.25%포인트 인상하면서 1500조원에 달하는 가계부채에 ...
2,2,1.0,0.2092,"금융, 한국은행, 서울, 기자, 지원, 회장, 서울_중구, 사업, 총재, 이주열",※ 저작권자
3,3,0.0,0.2000,"그룹, 은행, 상품, 고객, 기업, 영업, 업무, 보험, 우리은행, 통해",
4,4,1.0,0.2157,"금융, 한국은행, 서울, 기자, 지원, 회장, 서울_중구, 사업, 총재, 이주열",※ 저작권자
5,5,1.0,0.2157,"금융, 한국은행, 서울, 기자, 지원, 회장, 서울_중구, 사업, 총재, 이주열",※ 저작권자
6,6,1.0,0.2113,"금융, 한국은행, 서울, 기자, 지원, 회장, 서울_중구, 사업, 총재, 이주열",※ 저작권자
7,7,1.0,0.2135,"금융, 한국은행, 서울, 기자, 지원, 회장, 서울_중구, 사업, 총재, 이주열",※ 저작권자
8,8,0.0,0.5007,"그룹, 은행, 상품, 고객, 기업, 영업, 업무, 보험, 우리은행, 통해",모빌리티 블록체인 MVL(엠블)과 글로벌 보험사 AXA(악사)가 블록체인을 활용한...
9,9,2.0,0.5680,"금리, 인상, 기준금리, 포인트, 대출, 내년, 금융, 경기, 시장, 한국은행",【서울=뉴시스】조현아 기자 = 30일 한국은행이 1년 만에 금리인상 카드를 전격 꺼...


In [31]:
recommendation_df = df_dominant_topic.sort_values(by='Topic_Perc_Contrib',ascending=False)
recommendation_df.reset_index(drop=True,inplace=True)
recommendation_df

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,273,0.0,0.8231,"그룹, 은행, 상품, 고객, 기업, 영업, 업무, 보험, 우리은행, 통해",손태승<> 우리은행장이 우리금융지주(가칭) 회장 내정자로서의 첫 인사권을 행사하며...
1,495,0.0,0.8226,"그룹, 은행, 상품, 고객, 기업, 영업, 업무, 보험, 우리은행, 통해",우리은행은 내년 1월 지주사 설립을 위한 지주 임원 내정 및 은행 임원 인사를 단...
2,198,2.0,0.8151,"금리, 인상, 기준금리, 포인트, 대출, 내년, 금융, 경기, 시장, 한국은행",【서울=뉴시스】조현아 기자 = 30일 한국은행 금융통화위원회(금통위)의 금리인상이 ...
3,606,2.0,0.8148,"금리, 인상, 기준금리, 포인트, 대출, 내년, 금융, 경기, 시장, 한국은행",【서울=뉴시스】조현아 기자 = 30일 한국은행의 금리인상이 1년 만에 단행된 가운...
4,281,2.0,0.8139,"금리, 인상, 기준금리, 포인트, 대출, 내년, 금융, 경기, 시장, 한국은행",【서울=뉴시스】조현아 기자 = 30일 한국은행의 금리인상이 1년 만에 단행된 가운데...
5,585,2.0,0.8055,"금리, 인상, 기준금리, 포인트, 대출, 내년, 금융, 경기, 시장, 한국은행",【서울=뉴시스】조현아 기자 = 30일 한국은행 금융통화위원회(금통위)의 금리인상이...
6,583,4.0,0.7929,"카드, 중국, 기업, 산업, 정부, 업계, 미국, 시장, 경우, 기술",국세청은 올해 종합부동산세 납세의무자 46만6000명(세액 2조1148억원)에게...
7,236,0.0,0.7843,"그룹, 은행, 상품, 고객, 기업, 영업, 업무, 보험, 우리은행, 통해",손태승<>우리은행장이 우리금융지주(가칭) 회장 내정자로서의 첫 인사권을 행사하며 강...
8,381,0.0,0.7624,"그룹, 은행, 상품, 고객, 기업, 영업, 업무, 보험, 우리은행, 통해",우리은행은 내년 1월 성공적인 지주사 설립을 위한 지주 임원 내정 및 은행 임원 ...
9,261,3.0,0.7619,"증가, 상승, 국내, 하락, 투자, 성장, 올해, 지난, 경제, 지속",【서울=뉴시스】정리/천민아 기자 = 금융통화위원회는 다음 통화정책방향 결정시까지 한...


In [32]:
duplicate_ls = []
duplicate_idx = []
for idx,val in enumerate(recommendation_df['Dominant_Topic'].values):
    if val not in duplicate_ls:
        duplicate_ls.append(val)
        duplicate_idx.append(idx)
        if len(duplicate_ls) == 5 : break

In [40]:
display(recommendation_df.loc[duplicate_idx])
display(df.loc[recommendation_df.loc[duplicate_idx]['Document_No'].values])

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,432,1.0,0.7925,"금융, 은행, 투자, 통해, 회장, 한국, 산업, 성장, 경영, 사업",<아이뉴스24> 신한금융그룹은 26일 프랑스 파리에서 열린 유엔 환경계획 금융부문(...
7,212,2.0,0.7744,"카드, 롯데, 보험, 금융, 손해, 그룹, 매각, 수수료, 업계, 영업",<아이뉴스24> 롯데가 결국 금융업에서 철수한다는 뜻을 공식화하면서 롯데카드와 롯데...
11,687,0.0,0.7620,"정부, 미국, 거래, 증권, 삼성, 대한, 중국, 서울, 대해, 위원회",【서울=뉴시스】임종명 강지은 오제일 기자 =국회 예산결산특별위원회가 예산안조정소위 ...
24,594,3.0,0.7254,"금융, 기업, 테크, 관련, 회사, 포인트, 전망, 도입, 보험, 대출",(서울=연합뉴스) 김수현 기자 = 조선업 회복 영향으로 이번 달 기업 체감경기가 소...
73,676,4.0,0.6692,"고객, 서비스, 상품, 결제, 제공, 지역, 통해, 기자, 사용, 피해",공정거래위원회는 소비자 피해를 최소화하기 위해 상조업체에 자본금 증액을 압박하고 있...


,title,link,content
432,재계의 요청에도 공정거래법 개정안 국무회의서 의결,https://m.news.naver.com/read.nhn?oid=029&aid=...,중대·명백한 경성담합(가격담합 생산량 조절 시장분할 입찰담합 등 중대담합)에 대해서...
212,보험자본 건정성 선진화 추진단 1차 회의,https://m.news.naver.com/read.nhn?oid=421&aid=...,(서울=뉴스1) 이재명 기자 = 김용범 금융위원회 부위원장이 27일 오후 서울 세종...
687,"제주 삼다수 ""최악 피했다""..'생산공장 재가동' 결정(상보)",http://m.finance.daum.net/m/news/news_content....,(서울=뉴스1) 신건웅 기자 = 근로자 사망사고로 가동이 전면 중간됐던 제주개발공사...
594,"현대해상, '2018 스포트라이트 어워즈' 연차보고서 부문 금상 수상",https://m.news.naver.com/read.nhn?oid=015&aid=...,[ 차은지 기자] 현대해상은 '2018 스포트라이트 어워즈(Spotlight Awa...
676,"현대중 노조, 부당노동행위 항의 엿새째 파업..고용부 앞 집회",http://m.finance.daum.net/m/news/news_content....,(울산=연합뉴스) 김근주 기자 = 현대중공업 노조는 27일 사측 부당노동행위에 항의...


In [34]:
recommendation_df = df.loc[recommendation_df.loc[duplicate_idx]['Document_No'].values].iloc[:,:2]
recommendation_df

KeyError: 'None of [[0, 2, 6, 9, 40]] are in the [index]'

In [35]:
recommendation_df

,title,link
273,"금융안정에 무게 둔 한은, 1년 만에 기준금리 인상(종합)",https://m.news.naver.com/read.nhn?oid=421&aid=...
198,한은 기준금리 인상으로 가계·기업 대출 상환 부담 급증,https://m.news.naver.com/read.nhn?oid=421&aid=...
583,"순이익 급감한 한화생명, 내달부터 상시 희망퇴직 도입",https://m.news.naver.com/read.nhn?oid=015&aid=...
261,금리 인상에도 흔들림 없는 채권시장 '궁금증'…전문가 전망은,https://m.news.naver.com/read.nhn?oid=015&aid=...
162,"[전문] 이주열 한은 총재 ""기준금리 인상, 통화정책 기조는 완화""",https://m.news.naver.com/read.nhn?oid=417&aid=...


In [42]:
recommendation_df['link'].tolist()

['https://m.news.naver.com/read.nhn?oid=421&aid=0003718072&sid1=101&sid2=259&mode=LS2D',
 'https://m.news.naver.com/read.nhn?oid=421&aid=0003718286&sid1=101&sid2=259&mode=LS2D',
 'https://m.news.naver.com/read.nhn?oid=015&aid=0004053897&sid1=101&sid2=259&mode=LS2D',
 'https://m.news.naver.com/read.nhn?oid=015&aid=0004054318&sid1=101&sid2=259&mode=LS2D',
 'https://m.news.naver.com/read.nhn?oid=417&aid=0000360036&sid1=101&sid2=259&mode=LS2D']

# Telegram Service

In [44]:
import telegram # 텔레그램 모듈을 임포트합니다.

my_token = '717292107:AAGuwGQIlMnr-LlhtH5xqR4Zud8GyyKXbQY' # 토큰을 변수에 저장합니다.

bot = telegram.Bot(token = my_token) # bot을 선언합니다.

chat_id = bot.getUpdates()[-1].message.chat.id

bot.sendMessage(chat_id=768469872, text='안녕하세요. 좋은 아침입니다. {}토픽 별 기사 5개 추천해드리겠습니다.'.format(\
str(datetime.datetime.now().month)+' 월 ' + str(datetime.datetime.now().day)+' 일 '))

for news in recommendation_df['link'].tolist() :
    bot.sendMessage(chat_id = chat_id, text=news)

In [ ]:
# import telegram
# import pickle

# my_token = pickle.load( open('../../pw.pickle', 'rb'))
# my_token = my_token

# bot = telegram.Bot(token = my_token) #봇을 생성합니다.

# chat_id = bot.getUpdates()[-1].message.chat.id

# bot.sendMessage(chat_id=chat_id, text='안녕하세요. 좋은 아침입니다. {}토픽 별 기사 5개 추천해드리겠습니다.'.format(\
# str(datetime.datetime.now().month)+' 월 ' + str(datetime.datetime.now().day)+' 일 '))

# for news in recommendation_df['link'].tolist() :
#     bot.sendMessage(chat_id = chat_id, text=news)